In [1]:
# PRE-STEP: Install Required Dependencies
%pip install langchain
%pip install langgraph
%pip install langchain-community
%pip install langchain-openai
%pip install chromadb
%pip install python-dotenv


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 

In [2]:
# PRE-STEP: Build the Basic Agent
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.vectorstores import Chroma
from typing import TypedDict, Annotated, Sequence
from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from langgraph.graph import StateGraph, END

# Load environment variables
load_dotenv(dotenv_path='env.txt')
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

# Initialize models
llm = ChatOpenAI(model_name="gpt-4.1-mini", temperature=0)
embeddings = OpenAIEmbeddings()


# Define Agent State with memory placeholders
class AgentState(TypedDict):
    """State container for agent memory and messages"""
    messages: Annotated[Sequence[BaseMessage], add_messages]
    working_memory: dict  # Short-term context
    episodic_recall: list  # Retrieved past experiences
    semantic_facts: dict  # Retrieved knowledge

# Initialize vector store for future memory storage
vector_store = Chroma(
    collection_name="agent_memory",
    embedding_function=embeddings,
    persist_directory="./memory_store"
)

# Create base prompt and chain
base_prompt = PromptTemplate.from_template("""
You are a helpful assistant with memory capabilities.

Current conversation:
{messages}

Please respond to the latest message.
""")

output_parser = StrOutputParser()

# Define agent node
def agent_node(state: AgentState) -> dict:
    """Core agent logic - processes messages and generates responses"""
    messages = state["messages"][-5:] if state["messages"] else []
    formatted_messages = "\n".join([
        f"{msg.type}: {msg.content}" 
        for msg in messages
    ])
    
    chain = base_prompt | llm | output_parser
    response = chain.invoke({"messages": formatted_messages})
    
    return {"messages": [("assistant", response)]}

# Build and compile the graph
workflow = StateGraph(AgentState)
workflow.add_node("agent", agent_node)
workflow.set_entry_point("agent")
workflow.add_edge("agent", END)

app = workflow.compile()

/var/folders/pf/g8xt3fvd26n9h2z96v_vgnt80000gn/T/ipykernel_7279/4238200566.py:31: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vector_store = Chroma(


In [3]:
# PRE-STEP: Test the agent
test_input = {
    "messages": [("user", "Hello! What's the capital of France?")]
}

result = app.invoke(test_input)
print(result["messages"][-1].content)

Hello! The capital of France is Paris. How can I assist you further?


In [4]:
# Step 1: Import Additional Dependencies for Episodic Memory
from datetime import datetime
from typing import List
from langchain.schema import Document

In [5]:
# Step 2: Create Episodic Memory Storage Functions
def store_episodic_memory(vector_store, conversation_id: str, messages: List, summary: str = None):
    """Store a conversation episode in vector memory"""
    if not summary and messages:
        summary = f"Conversation about: {messages[0].content[:100]}..."
    
    doc = Document(
        page_content="\n".join([f"{msg.type}: {msg.content}" for msg in messages]),
        metadata={
            "type": "episodic",
            "conversation_id": conversation_id,
            "timestamp": datetime.now().isoformat(),
            "message_count": len(messages)
        }
    )
    vector_store.add_documents([doc])
    return conversation_id

def retrieve_episodic_memories(vector_store, query: str, k: int = 3):
    """Retrieve relevant past conversation episodes"""
    return vector_store.similarity_search(query, k=k, filter={"type": "episodic"})

In [6]:
# Step 3: Enhance Agent Node with Episodic Recall
def agent_with_episodic_memory(state: AgentState) -> dict:
    """Agent that retrieves and uses episodic memories"""
    messages = state.get("messages", [])
    
    # Retrieve relevant memories
    past_episodes = retrieve_episodic_memories(vector_store, messages[-1].content, k=2) if messages else []
    episodic_context = ("Relevant past conversations:\n" + "\n".join(
        f"\n[{ep.metadata.get('timestamp', 'Unknown')}]:\n{ep.page_content[:200]}..."
        for ep in past_episodes
    )) if past_episodes else ""
    
    # Generate response
    response = (PromptTemplate.from_template("""
You are a helpful assistant with episodic memory of past conversations.

{episodic_context}

Current conversation:
{messages}

Please respond to the latest message, utilizing relevant past conversations if helpful.
""") | llm | output_parser).invoke({
        "episodic_context": episodic_context,
        "messages": "\n".join(f"{m.type}: {m.content}" for m in messages[-5:]) if messages else ""
    })
    
    return {"messages": [("assistant", response)], "episodic_recall": past_episodes}

In [ ]:
# Step 4: Build Memory-Enabled Workflow
def store_conversation_node(state: AgentState) -> dict:
    """Store the current conversation as an episodic memory"""
    messages = state.get("messages", [])
    if len(messages) >= 2:  # Only store meaningful conversations
        store_episodic_memory(vector_store, f"conv_{datetime.now().timestamp()}", messages)
    return {"working_memory": {"stored": True}}

# Create workflow with episodic memory
memory_workflow = StateGraph(AgentState)
memory_workflow.add_node("recall_and_respond", agent_with_episodic_memory)
memory_workflow.add_node("store_memory", store_conversation_node)
memory_workflow.set_entry_point("recall_and_respond")
memory_workflow.add_edge("recall_and_respond", "store_memory")
memory_workflow.add_edge("store_memory", END)

memory_app = memory_workflow.compile()

In [8]:
# Step 5: Test Episodic Memory Functionality
# First conversation - store a memory
result_1 = memory_app.invoke({
    "messages": [("user", "I'm planning a trip to Paris next month. Any recommendations?")]
})
print("First conversation response:")
print(result_1["messages"][-1].content)
print("\n" + "="*50 + "\n")

# Second conversation - should recall the Paris discussion
result_2 = memory_app.invoke({
    "messages": [("user", "What are some good restaurants in Paris?")]
})
print("Second conversation response (with episodic recall):")
print(result_2["messages"][-1].content)

if result_2.get("episodic_recall"):
    print(f"\nRecalled {len(result_2['episodic_recall'])} relevant memories")

First conversation response:
That sounds wonderful! Paris is a fantastic city with so much to offer. Here are some recommendations for your trip:

1. **Must-See Attractions:**
   - **Eiffel Tower:** Iconic and a must-visit. Consider booking tickets in advance to avoid long lines.
   - **Louvre Museum:** Home to the Mona Lisa and countless other masterpieces.
   - **Notre-Dame Cathedral:** Beautiful Gothic architecture (check current status for restoration updates).
   - **Montmartre & Sacré-Cœur:** Charming neighborhood with great views of the city.
   - **Champs-Élysées & Arc de Triomphe:** Perfect for a stroll and some shopping.

2. **Cultural Experiences:**
   - Take a Seine River cruise, especially at sunset.
   - Visit the Musée d'Orsay for Impressionist art.
   - Explore Le Marais district for trendy shops, cafes, and historic sites.

3. **Food & Drink:**
   - Try classic French pastries like croissants, macarons, and éclairs.
   - Enjoy a meal at a traditional bistro or café.
  